![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Running multiple ML models at the same time
This notebook is an example of how to run multiple models side-by-side and combine results of all models. A video stream from a video source is processed by the hand, face, and person detection models. Combined result is then displayed.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `inference_host_address` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want to run your inferences, model_zoo_url, model names for hand, face, and person detection, and video source

In [ ]:
# inference_host_address: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# hand_det_model_name: name of the model for hand detection
# face_det_model_name: name of the model for face detection
# person_det_model_name: name of the model for person detection
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
inference_host_address = "@cloud"
face_det_model_zoo_url = "degirum/public"
face_det_model_name = "yolo_v5s_face_det--512x512_quant_n2x_orca1_1"
hand_det_model_zoo_url = "degirum/public"
hand_det_model_name = "yolo_v5s_hand_det--512x512_quant_n2x_orca1_1"
person_det_model_zoo_url = "degirum/public"
person_det_model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/person_face_hand.mp4"
overlay_color=[(255,255,0), (0,255,0), (255,0,0)]

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# Load face detection, hand detection models, and person detection models
face_det_model = dg.load_model(
    model_name=face_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=face_det_model_zoo_url,
    token=degirum_tools.get_token(),
)
hand_det_model = dg.load_model(
    model_name=hand_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=hand_det_model_zoo_url,
    token=degirum_tools.get_token(),
)
person_det_model = dg.load_model(
    model_name=person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=person_det_model_zoo_url,
    token=degirum_tools.get_token(),
)
person_det_model.overlay_color=overlay_color

# Create a compound model that combines the three models
combined_model=degirum_tools.CombiningCompoundModel(
    degirum_tools.CombiningCompoundModel(hand_det_model, face_det_model),
    person_det_model,
)

# run AI inference on video stream
inference_results=degirum_tools.predict_stream(combined_model,video_source)

# display inference results
# Press 'x' or 'q' to stop
with degirum_tools.Display("Hands, Faces and Persons") as display:
    for inference_result in inference_results:
        display.show(inference_result)